In [3]:
import numpy as np 
import pandas as pd
import math

data=pd.read_csv('seeds_dataset.csv')
data.head()

,A,B,C,D,E,F,G,T
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [4]:
def normalise_scale(y,a,b):
    l=[]
    for i in range(len(y)):
        l.append(((y[i]-min(y))/(max(y)-min(y)))*(a-b)+b)
        
    return l  

target=np.asarray(data['T'])
data=np.asarray(data.drop('T',axis=1))
for i in range(len(data[0])):
    data[:,i]=normalise_scale(data[:,i],0.999,.001)

print(data)

[[0.44110009 0.50206198 0.57063884 ... 0.48612901 0.18992304 0.34545987]
 [0.40528895 0.44638843 0.66210708 ... 0.501067   0.03381725 0.21573461]
 [0.34968744 0.34741322 0.87855172 ... 0.50391233 0.25195011 0.15136337]
 ...
 [0.24696601 0.25874793 0.72731216 ... 0.42922238 0.98070331 0.26487297]
 [0.11879981 0.16595868 0.3994755  ... 0.14753457 0.36860772 0.25897637]
 [0.16215014 0.19276446 0.54709256 ... 0.2456985  0.633196   0.26831265]]


In [5]:
def cost_func(ypred,y):
    return -np.sum(np.asarray([math.log(ypred[x],(2)) if y[x]==1 else 0 for x in range(len(y)) ]))

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z))

def sigmoid(z):
    sig=1/(1+np.exp(-z))
    sig=np.minimum(sig, 0.9999)
    sig=np.maximum(sig, 0.0001)
    return sig

def initialize_weights(inp_neurons,out_neurons):
    l=[]
    for i in range(out_neurons):
        W=np.random.randn(inp_neurons)
        l.append(W)
    b=np.zeros(out_neurons)
    return l,b

def normalise_scale(y,a,b):
    l=[]
    for i in range(len(y)):
        l.append(((y[i]-min(y))/(max(y)-min(y)))*(a-b)+b)
        
    return l  
    
def forwardpropogate(totalW,totalB,data,n):
    ffout=[]
    ffout.append(data)
    for i in range(0,n-1):
        params=[]
        for j in range(len(totalW[i])):
            
            ###############  W(transpose).X + B  ###############
            
            params.append(np.dot(np.asarray(totalW[i][j]).reshape(1,len(totalW[i][j])),np.asarray(data).reshape(len(data),1)).reshape(1)[0]+totalB[i][j])
        
        data=params
        if(i!=n-2):
            ##################  activated  ###############
    
            data=sigmoid(np.asarray(data))
            ffout.append(data)
        
    ##################  final_layer-activated  ###############
    
    ffout.append(softmax(np.asarray(data)))
    return ffout

def backwardpropogate(totalW,ffout,y,n):
    totalgradW=[]
    totalgradB=[]
    
    ##################  final_layer-gradient  ###############
    
    grad_wrt_out=-(np.asarray(y)-np.asarray(ffout[len(ffout)-1]))
    grad_inter=grad_wrt_out
    
    for i in range(n-2,-1,-1):
        
        ##################  gradW = (gradient_at_layer) . (activated_output_previous_layer)  ###############
        
        totalgradW.insert(0,np.dot(np.asarray(grad_inter).reshape(len(grad_inter),1),np.asarray(ffout[i]).reshape(1,len(ffout[i]))).reshape(len(grad_inter),len(ffout[i])))
        ##################  gradB = (gradient_at_layer)  ###############
        
        totalgradB.insert(0,grad_inter)
        grad_pre=[]
        
        ##################  grad_at_previous_layer = (gradient_at_layer) . (weight_matrix)  ###############
        
        for j in range(len(totalW[i][0])):
            grad_pre.append(np.dot(np.asarray(grad_inter).reshape(1,len(grad_inter)),np.asarray(np.matrix(totalW[i])[:,j])).reshape(1)[0])
        
        ####  grad_at_previous_layer = (grad_at_previous_layer) . (hadamard) . (derivative_of_activation)  ####
        
        grad_inter=np.asarray(grad_pre)*(np.asarray(ffout[i]))*(1-np.asarray(ffout[i]))
    
    return totalgradW,totalgradB
                   
    
totalW=[]
totalB=[]
max_iterations=50
w,b=initialize_weights(len(data[0]),5)
totalW.append(w)
totalB.append(b)
w,b=initialize_weights(5,3)
totalW.append(w)
totalB.append(b)
#w,b=initialize_weights(3,3)
#totalW.append(w)
#totalB.append(b)
no_layers=3

cnt=0
while(cnt<max_iterations):
    cnt+=1
    for i in range(len(data)):
        y_one_hot=[1 if x==target[i] else 0 for x in set(target)]
        print(y_one_hot)
     
        ffout=forwardpropogate(totalW,totalB,data[i],no_layers)
        C=cost_func(ffout[len(ffout)-1],y_one_hot)
        print('loss=======',C)
        print('value obtained :',ffout[len(ffout)-1])
        
        gradW,gradB=backwardpropogate(totalW,ffout,y_one_hot,no_layers)
        
        if(i==0):
            totalgradW1=gradW
            totalgradB1=gradB
        else:
            for j in range(len(gradW)):
                totalgradW1[j]=np.asarray(totalgradW1[j])+np.asarray(gradW[j])
                totalgradB1[j]=np.asarray(totalgradB1[j])+np.asarray(gradB[j])
        if((i+1) % 32 == 0):
            for i in range(len(totalW)):
                totalW[i]=np.asarray(totalW[i])-(0.05*np.asarray(totalgradW1[i]))
                totalB[i]-=0.05*totalgradB1[i]
            
            for j in range(len(gradW)):
                totalgradW1[j]=0
                totalgradB1[j]=0
                
            print()

    print()
    

[1, 0, 0]
loss======= 4.370574871478958
value obtained : [0.04834214 0.04111089 0.91054697]
[1, 0, 0]
loss======= 4.0707437080250335
value obtained : [0.05950919 0.05122988 0.88926093]
[1, 0, 0]
loss======= 3.8670605278416925
value obtained : [0.06853285 0.05244511 0.87902204]
[1, 0, 0]
loss======= 3.7149142682749616
value obtained : [0.07615517 0.05782923 0.8660156 ]
[1, 0, 0]
loss======= 4.448787012453271
value obtained : [0.04579116 0.04448369 0.90972515]
[1, 0, 0]
loss======= 4.003120318452585
value obtained : [0.06236497 0.04891641 0.88871862]
[1, 0, 0]
loss======= 4.271954815910435
value obtained : [0.05176229 0.03868898 0.90954874]
[1, 0, 0]
loss======= 3.982742318088781
value obtained : [0.06325212 0.04905889 0.88768899]
[1, 0, 0]
loss======= 5.059271160555052
value obtained : [0.02999215 0.02960293 0.94040492]
[1, 0, 0]
loss======= 4.764822867357622
value obtained : [0.03678285 0.03589857 0.92731858]
[1, 0, 0]
loss======= 4.392256481443667
value obtained : [0.04762106 0.033858

loss======= 0.08241265579659819
value obtained : [0.04528998 0.94447685 0.01023316]
[0, 1, 0]
loss======= 0.07679665852356328
value obtained : [0.04341128 0.94816059 0.00842813]
[0, 1, 0]
loss======= 0.08828498643924491
value obtained : [0.04640766 0.94064028 0.01295207]
[0, 1, 0]
loss======= 0.08651899132179326
value obtained : [0.04701372 0.94179242 0.01119387]
[0, 1, 0]
loss======= 0.09383260386631531
value obtained : [0.05166372 0.93703016 0.01130612]
[0, 1, 0]
loss======= 0.10439995080242283
value obtained : [0.06035819 0.93019175 0.00945006]
[0, 1, 0]
loss======= 0.07436874699046046
value obtained : [0.04314834 0.9497576  0.00709406]
[0, 1, 0]
loss======= 0.08468827170934135
value obtained : [0.05308787 0.94298827 0.00392386]
[0, 1, 0]
loss======= 0.0752882903447583
value obtained : [0.0431745  0.94915244 0.00767306]
[0, 1, 0]
loss======= 0.09683537414228728
value obtained : [0.05169253 0.93508189 0.01322558]
[0, 1, 0]
loss======= 0.07805767914693429
value obtained : [0.04504904 

[1, 0, 0]
loss======= 5.464033983312523
value obtained : [0.02265489 0.03538158 0.94196354]
[1, 0, 0]
loss======= 5.435281286851097
value obtained : [0.02311092 0.03488838 0.94200069]
[1, 0, 0]
loss======= 5.30164527389669
value obtained : [0.02535396 0.03851207 0.93613397]
[1, 0, 0]
loss======= 5.795610090925619
value obtained : [0.01800311 0.03260857 0.94938832]
[1, 0, 0]
loss======= 5.564981608574628
value obtained : [0.02112388 0.0317766  0.94709952]
[1, 0, 0]
loss======= 6.0050435281159675
value obtained : [0.01557047 0.02293049 0.96149904]
[1, 0, 0]
loss======= 5.606703991384132
value obtained : [0.02052173 0.03080768 0.94867059]
[1, 0, 0]
loss======= 6.376244459322165
value obtained : [0.01203813 0.0207573  0.96720457]
[1, 0, 0]
loss======= 6.148383040997466
value obtained : [0.01409783 0.02499298 0.96090919]
[1, 0, 0]
loss======= 6.225033989533503
value obtained : [0.01336836 0.01981511 0.96681653]
[1, 0, 0]
loss======= 5.479826451612421
value obtained : [0.02240825 0.03179034 

[0, 0, 1]
loss======= 1.029295524605556
value obtained : [0.17705349 0.33299718 0.48994934]
[0, 0, 1]
loss======= 1.217699877515502
value obtained : [0.1945391  0.37549322 0.42996768]
[0, 0, 1]
loss======= 1.3189501132896309
value obtained : [0.22739653 0.37177695 0.40082652]
[0, 0, 1]
loss======= 1.160997791671658
value obtained : [0.2008441  0.35195277 0.44720314]
[0, 0, 1]
loss======= 1.0495401737728152
value obtained : [0.19510291 0.32177497 0.48312212]
[0, 0, 1]
loss======= 1.386180083976516
value obtained : [0.23256771 0.38485586 0.38257643]
[0, 0, 1]
loss======= 1.170637072610615
value obtained : [0.20278751 0.35298736 0.44422513]
[0, 0, 1]
loss======= 1.2691823098889852
value obtained : [0.23139788 0.35370726 0.41489486]
[0, 0, 1]
loss======= 1.2350634760361432
value obtained : [0.20162381 0.37355239 0.42482381]
[0, 0, 1]
loss======= 1.5278226024643866
value obtained : [0.21085415 0.44234547 0.34680038]
[0, 0, 1]
loss======= 1.4338442730022474
value obtained : [0.23283871 0.397

[0, 1, 0]
loss======= 0.8201068258068164
value obtained : [0.35441393 0.5664     0.07918607]
[0, 1, 0]
loss======= 0.8146812958230456
value obtained : [0.34124329 0.56853407 0.09022264]
[0, 1, 0]
loss======= 0.8157502244269026
value obtained : [0.34842097 0.56811298 0.08346605]
[0, 1, 0]
loss======= 0.8025085502119815
value obtained : [0.32933328 0.57335137 0.09731535]
[0, 1, 0]
loss======= 0.8002042271293873
value obtained : [0.31860262 0.57426788 0.1071295 ]
[0, 1, 0]
loss======= 0.8623456252968198
value obtained : [0.3512905  0.55005751 0.09865199]
[0, 1, 0]
loss======= 0.9591196601686548
value obtained : [0.38059912 0.51437069 0.10503019]
[0, 1, 0]
loss======= 0.8283078928345252
value obtained : [0.3612103  0.56318941 0.07560029]
[0, 1, 0]
loss======= 0.9583286566216854
value obtained : [0.42358029 0.51465279 0.06176693]
[0, 1, 0]
loss======= 0.8150110647408952
value obtained : [0.34614382 0.56840413 0.08545206]
[0, 1, 0]
loss======= 0.838968133811784
value obtained : [0.33378621 0

...

[1, 0, 0]
loss======= 0.01205841769756914
value obtained : [9.91676575e-01 6.72821396e-05 8.25614296e-03]
[1, 0, 0]
loss======= 0.25259419890818297
value obtained : [8.39385706e-01 6.34970732e-04 1.59979323e-01]

[1, 0, 0]
loss======= 0.005217697894822419
value obtained : [9.96389900e-01 5.14198003e-05 3.55868065e-03]
[1, 0, 0]
loss======= 0.003560905069651371
value obtained : [9.97534812e-01 4.28599587e-05 2.42232777e-03]
[1, 0, 0]
loss======= 0.00391296942944988
value obtained : [9.97291411e-01 3.56338354e-04 2.35225051e-03]
[1, 0, 0]
loss======= 0.010917548736177176
value obtained : [9.92461093e-01 4.92827074e-04 7.04607985e-03]
[1, 0, 0]
loss======= 0.00678092787159467
value obtained : [9.95310848e-01 5.26114203e-04 4.16303827e-03]
[1, 0, 0]
loss======= 0.8428588909781888
value obtained : [5.57537639e-01 2.59143853e-04 4.42203217e-01]
[0, 1, 0]
loss======= 0.07993594855996783
value obtained : [5.37158793e-02 9.46099650e-01 1.84470872e-04]
[0, 1, 0]
loss======= 0.17447681624150174
v

[1, 0, 0]
loss======= 0.3348663256629124
value obtained : [0.7928576  0.16031661 0.04682579]
[1, 0, 0]
loss======= 0.03525788738655705
value obtained : [0.97585731 0.00852679 0.01561591]
[1, 0, 0]
loss======= 6.067841759678076
value obtained : [1.49072530e-02 9.84991835e-01 1.00911585e-04]
[1, 0, 0]
loss======= 3.835696095563351
value obtained : [7.00390781e-02 9.29763975e-01 1.96947183e-04]
[1, 0, 0]
loss======= 2.160883099089706
value obtained : [0.22361934 0.73191416 0.04446649]
[1, 0, 0]
loss======= 0.03571196577190005
value obtained : [0.97555021 0.00545369 0.0189961 ]
[1, 0, 0]
loss======= 0.059069655256748156
value obtained : [0.95988292 0.0041428  0.03597429]
[1, 0, 0]
loss======= 0.08799836034731626
value obtained : [0.94082718 0.00585333 0.05331949]
[1, 0, 0]
loss======= 0.08282083478591785
value obtained : [0.94420967 0.00406363 0.0517267 ]
[1, 0, 0]
loss======= 0.03354368265803662
value obtained : [0.97701751 0.00801574 0.01496676]
[1, 0, 0]
loss======= 0.03822104728652801


value obtained : [0.08128359 0.91651977 0.00219664]
[0, 1, 0]
loss======= 0.21949224992754168
value obtained : [0.13922607 0.85886766 0.00190628]
[0, 1, 0]
loss======= 0.0627473917658902
value obtained : [4.23860127e-02 9.57439086e-01 1.74901279e-04]
[0, 0, 1]
loss======= 0.07616908518023215
value obtained : [0.04812689 0.00329998 0.94857313]
[0, 0, 1]
loss======= 0.058707093558533784
value obtained : [0.03072355 0.00915227 0.96012417]
[0, 0, 1]
loss======= 0.19536749588008998
value obtained : [0.12074602 0.00590358 0.8733504 ]
[0, 0, 1]
loss======= 0.1355498912830469
value obtained : [8.91500048e-02 5.27202320e-04 9.10322793e-01]
[0, 0, 1]
loss======= 0.016181908231902586
value obtained : [1.10577966e-02 9.59777099e-05 9.88846226e-01]
[0, 0, 1]
loss======= 0.0016717743413061828
value obtained : [1.12657523e-03 3.15393048e-05 9.98841885e-01]
[0, 0, 1]
loss======= 0.22358918566429195
value obtained : [1.43409527e-01 1.58350108e-04 8.56432123e-01]
[0, 0, 1]
loss======= 1.4813295661192165

[1, 0, 0]
loss======= 0.49466404412835074
value obtained : [0.70972693 0.28879751 0.00147556]
[1, 0, 0]
loss======= 0.4609072823179377
value obtained : [7.26529215e-01 2.73181183e-01 2.89601753e-04]
[1, 0, 0]
loss======= 0.016442653851848947
value obtained : [0.98866752 0.00540578 0.0059267 ]
[1, 0, 0]
loss======= 0.09459723383232427
value obtained : [0.93653367 0.00202455 0.06144178]
[1, 0, 0]
loss======= 0.014917642135911053
value obtained : [9.89713154e-01 3.10711742e-04 9.97613457e-03]
[1, 0, 0]
loss======= 0.01265680720558925
value obtained : [9.91265341e-01 2.81220980e-04 8.45343851e-03]
[1, 0, 0]
loss======= 0.004498900279572037
value obtained : [9.96886457e-01 8.22052378e-05 3.03133765e-03]
[1, 0, 0]
loss======= 0.5102523905837113
value obtained : [0.7020996  0.29103312 0.00686729]
[1, 0, 0]
loss======= 0.006252320928914363
value obtained : [0.9956756  0.00276727 0.00155713]
[1, 0, 0]
loss======= 0.0060685973630997
value obtained : [9.95802403e-01 1.74653730e-04 4.02294278e-03]

loss======= 0.002067299825675209
value obtained : [1.34843004e-03 8.34868322e-05 9.98568083e-01]
[0, 0, 1]
loss======= 0.014310295088774806
value obtained : [9.61334998e-03 2.56758289e-04 9.90129892e-01]
[0, 0, 1]
loss======= 0.05196180703122845
value obtained : [3.53210457e-02 5.52332659e-05 9.64623721e-01]
[0, 0, 1]
loss======= 0.020380858118929476
value obtained : [1.39641072e-02 6.35102243e-05 9.85972383e-01]
[0, 0, 1]
loss======= 0.002550619457448228
value obtained : [1.73987468e-03 2.65180967e-05 9.98233607e-01]
[0, 0, 1]
loss======= 0.8246429357970333
value obtained : [4.35141619e-01 2.36453421e-04 5.64621928e-01]
[0, 0, 1]
loss======= 0.004374856590023524
value obtained : [2.94122004e-03 8.66063290e-05 9.96972174e-01]
[0, 0, 1]
loss======= 0.0035970544531511084
value obtained : [2.42161746e-03 6.85650269e-05 9.97509818e-01]
[0, 0, 1]
loss======= 0.0030944125874920684
value obtained : [2.09962940e-03 4.29553374e-05 9.97857415e-01]

[0, 0, 1]
loss======= 0.05077907281326507
value

In [7]:
ffout=forwardpropogate(totalW,totalB,data[200],no_layers)
y_one_hot=[1 if x==target[200] else 0 for x in set(target)]
print('Expected Output :',y_one_hot)
print('Predicted Output :',ffout[len(ffout)-1])

Expected Output : [0, 0, 1]
Predicted Output : [1.43623891e-02 2.09577283e-04 9.85428034e-01]
